In [16]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [17]:
data = TextLoader("text.txt")
data = loader = data.load()
data


[Document(metadata={'source': 'text.txt'}, page_content='Solar energy is one of the cleanest and most abundant renewable energy sources available. Unlike fossil fuels, solar power does not release harmful greenhouse gases into the atmosphere when generating electricity. Solar panels, typically installed on rooftops or in large solar farms, capture sunlight and convert it into usable energy through photovoltaic cells. While the initial installation cost can be high, prices have dropped significantly over the past decade, making solar energy more accessible to homeowners and businesses. Additionally, many governments offer tax incentives to encourage the adoption of solar technology as part of broader efforts to combat climate change.')]

In [18]:
#split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)
splits

[Document(metadata={'source': 'text.txt'}, page_content='Solar energy is one of the cleanest and most abundant renewable energy sources available. Unlike fossil fuels, solar power does not release harmful greenhouse gases into the atmosphere when generating electricity. Solar panels, typically installed on rooftops or in large solar farms, capture sunlight and convert it into usable energy through photovoltaic cells. While the initial installation cost can be high, prices have dropped significantly over the past decade, making solar energy more'),
 Document(metadata={'source': 'text.txt'}, page_content='accessible to homeowners and businesses. Additionally, many governments offer tax incentives to encourage the adoption of solar technology as part of broader efforts to combat climate change.')]

In [19]:
embedding = OllamaEmbeddings(model="gemma:2b")
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
vectordb

In [22]:
#query it
query = "What is solar energy considered to be?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'accessible to homeowners and businesses. Additionally, many governments offer tax incentives to encourage the adoption of solar technology as part of broader efforts to combat climate change.'

In [23]:
#saving on the disk
vectordb=Chroma.from_documents(documents=splits, embedding=embedding, persist_directory="./chroma_db")

In [ ]:
#load from disk
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embedding)
docs=db2.similarity_search(query)
print(docs[0].page_content)

accessible to homeowners and businesses. Additionally, many governments offer tax incentives to encourage the adoption of solar technology as part of broader efforts to combat climate change.


In [25]:
docs = vectordb.similarity_search_with_score(query)
docs

[(Document(id='b856e7f7-bded-42db-91d7-e65cfe48af6b', metadata={'source': 'text.txt'}, page_content='accessible to homeowners and businesses. Additionally, many governments offer tax incentives to encourage the adoption of solar technology as part of broader efforts to combat climate change.'),
  2293.77978515625),
 (Document(id='03c5d1f0-9c3f-4d65-a734-28f305cfc1e1', metadata={'source': 'text.txt'}, page_content='Solar energy is one of the cleanest and most abundant renewable energy sources available. Unlike fossil fuels, solar power does not release harmful greenhouse gases into the atmosphere when generating electricity. Solar panels, typically installed on rooftops or in large solar farms, capture sunlight and convert it into usable energy through photovoltaic cells. While the initial installation cost can be high, prices have dropped significantly over the past decade, making solar energy more'),
  5904.8994140625)]

In [26]:
#retriever option
retriever = vectordb.as_retriever()
retriever.invoke(query)[0].page_content

'accessible to homeowners and businesses. Additionally, many governments offer tax incentives to encourage the adoption of solar technology as part of broader efforts to combat climate change.'